In [18]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
import sklearn
import optuna
from scipy.stats import bootstrap as boot
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
sns.set_theme()

c:\Users\unstr\AppData\Local\R-MINI~1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('C:/Users/unstr/Documents/6 семестр работы/Отрынки/mortgage_project_2024/arctic1009.csv')
df = df.rename(columns = {'date': 'month_dt'})
df

,city,population,p_avg_per_m_sec,month_dt,t,dummy_space,dummy_time,reg_russian,federal_district,lat,lon,city_id,city_rk
0,Барнаул,630877,81073.0,2022-01-01,1,0,0,Алтайский край,Сибирский,53.347361,83.77833,184,Барнаул_Алтайский край
1,Нижнеудинск,29995,52594.0,2022-01-01,1,0,0,Иркутская область,Сибирский,54.900000,99.01667,820,Нижнеудинск_Иркутская область
2,Верхняя Пышма,71335,77425.0,2022-01-01,1,0,0,Свердловская область,Уральский,56.966670,60.58333,152,Верхняя Пышма_Свердловская область
3,Бирюсинск,8632,53207.0,2022-01-01,1,0,0,Иркутская область,Сибирский,55.950000,97.81667,150,Бирюсинск_Иркутская область
4,Арамиль,19013,68613.0,2022-01-01,1,0,0,Свердловская область,Уральский,56.700000,60.83333,793,Арамиль_Свердловская область
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27542,Ипатово,26122,54183.0,2024-08-01,32,0,1,Ставропольский край,Северо-Кавказский,45.716670,42.90000,272,Ипатово_Ставропольский край
27543,Белая Холуница,9659,15444.0,2024-08-01,32,0,1,Кировская область,Приволжский,58.833330,50.85000,326,Белая Холуница_Кировская область
27544,Беломорск,7708,42477.0,2024-08-01,32,1,1,Республика Карелия,Северо-Западный,64.516670,34.76667,887,Беломорск_Республика Карелия
27545,Пудож,7356,44961.0,2024-08-01,32,0,1,Республика Карелия,Северо-Западный,61.800000,36.53333,811,Пудож_Республика Карелия


In [3]:
df = df[df['month_dt'] >= '2022-01-01']

In [4]:
x = df[df['month_dt'] >= '2022-01-01']
x = pd.pivot_table(x, index = 'city_id', aggfunc="count").reset_index()

x = x[['city_id', 'city']].rename(columns = {'city': 'count'})
x

x1 = x[x['count'] == 32]
x1


,city_id,count
0,0,32
2,2,32
3,3,32
5,5,32
6,6,32
...,...,...
919,946,32
920,947,32
921,948,32
922,949,32


In [5]:
#сколько было тритмента
treat = df.loc[df['dummy_space'] == 1, 'city_id']
treat = treat.drop_duplicates()
treat =  treat.to_list()
len(treat)

#сколько осталось тритмента
city_id = x1['city_id'].to_list()
df = df[df['city_id'].isin(city_id)]
len(df.loc[df['dummy_space'] == 1, 'city_id'].unique())

22

In [6]:
control_cities = list(set(list(df.query('dummy_space== 0')['city_id'])))
len(control_cities)

643

In [7]:
#Создаю зависимую переменную в процентах к ноябрю 2023 года
tmp = df.query('month_dt == "2023-11-01"')
price_dict = {tmp['city_id'].iloc[i] :tmp['p_avg_per_m_sec'].iloc[i] for i in range(len(tmp))}
df['price_scaled'] = df['p_avg_per_m_sec']/df['city_id'].apply(lambda x: price_dict[x])

In [9]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from datetime import datetime
from tqdm import tqdm


#Берём один город, на городах контрольной группы (кроме заданного города), считаем ATE для него.Проделываем операцию для всех городов.

def synth_ci_new(df
    , alpha:float
    , id_name = 'city_rk' 
    , t_name = 'month_dt'
    , dummy_space_name = 'dummy_space' 
    , t_treatment = '2023-12-01'
    , y_name = 'p_avg_per_m_sec'
    , trend = False
    ):
    
    df[id_name] = df[id_name].astype(str)
    treatment_cities = list(set(list(df.query(dummy_space_name+'== 1')[id_name])))
    control_cities = list(set(list(df.query(dummy_space_name+'== 0')[id_name])))
    cities_list = list(set(list(df[id_name])))
    def sc(city_id):
        '''Функция, которая принимает на вход id города 
        и выдаёт временной ряд среднего эффекта воздействия'''

        tmp = df[df[id_name].apply(lambda x:(x in control_cities) or (x == city_id))]
        #Вытягиваем в столбик: цена в городе city цены в тот же период в городах контрольной группы
        tmp = tmp.pivot(index = t_name, columns = id_name, values = y_name).reset_index()

        tmp['tr_city'] = tmp[city_id] #Зависимая переменная 
        tmp.drop(columns = city_id, inplace = True)
        
        if trend:
            #Разница в датах считается в днях, потому что по-другому я не смог
            tmp['t'] = tmp[t_name].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))-datetime.strptime('2022-04-01', '%Y-%m-%d')
            tmp['t'] = tmp['t'].apply(lambda x: abs(x).days)  

        #Обучаем регрессор на претренде
        train = tmp.loc[tmp[t_name] < t_treatment]
        #Не забываем отшкалировать переменные, чтобы корректино предсказывать 
        scaler = StandardScaler()
        lr = Lasso(alpha=alpha)
        lr.fit(scaler.fit_transform(train.drop(columns = ['tr_city', t_name])), train['tr_city'])
            #Записываем в результат разницу между истинным значением и предиктом
        res = tmp['tr_city']-lr.predict(scaler.transform(tmp.drop(columns = ['tr_city', t_name])))
        return res
    
    res = {city_id:sc(city_id) for city_id in tqdm(cities_list)}
    #Рассчитывать время, строить две вертикальные прямые. Использовать зависимую переменную в процентах (за 100% будем брать цену в городе в ноябре 2023 года)

    return pd.DataFrame(res)

In [12]:
t_treatment= '2023-06-01'
data = synth_ci_new(df, 0.1, t_treatment= t_treatment, trend=True, y_name='price_scaled')
data

100%|██████████| 665/665 [01:34<00:00,  7.01it/s]


,Нижняя Тура_Свердловская область,Апшеронск_Краснодарский край,Горнозаводск_Пермский край,Карпинск_Свердловская область,Кудымкар_Пермский край,Агидель_Республика Башкортостан,Боровск_Калужская область,Липецк_Липецкая область,Кингисепп_Ленинградская область,Тейково_Ивановская область,...,Майкоп_Республика Адыгея,Давлеканово_Республика Башкортостан,Карабаново_Владимирская область,Петергоф_Санкт-Петербург,Славгород_Алтайский край,Гороховец_Владимирская область,Чаплыгин_Липецкая область,Радужный_Владимирская область,Талица_Свердловская область,Канаш_Чувашская Республика
0,-0.118031,-0.193937,-0.043256,-0.098912,-0.156373,-0.079696,-0.181285,-0.043641,0.120762,-0.216660,...,-0.045956,0.017220,-0.150569,-0.073689,-0.075422,-0.034642,-0.226464,-0.079465,-0.088448,-0.069288
1,-0.117203,-0.054077,0.059440,-0.077803,-0.062629,-0.098279,-0.152463,-0.029033,0.176553,-0.189798,...,-0.033226,-0.009292,-0.163973,-0.030750,-0.067734,0.019358,0.128095,-0.073260,-0.082799,-0.086197
2,-0.056158,-0.000191,0.004365,-0.087247,-0.090615,-0.086311,-0.056780,0.022524,0.148652,-0.121841,...,-0.000048,0.003064,-0.140028,0.010404,-0.014126,0.063588,0.006265,-0.059652,-0.064439,-0.077046
3,-0.007675,0.015878,0.023359,-0.050361,-0.073962,0.003354,-0.039388,0.023782,0.115674,-0.046673,...,0.020361,0.033532,-0.092037,0.030222,-0.003619,-0.042930,-0.103174,-0.008029,-0.069065,-0.065303
4,0.014548,-0.001082,0.014168,-0.078317,-0.005694,-0.082373,-0.009711,0.010396,0.096640,-0.074047,...,0.022350,-0.006492,-0.113160,0.018835,0.007246,-0.071761,-0.190363,-0.026193,-0.050623,-0.034411
5,0.044156,-0.002975,-0.087064,-0.079345,0.042812,-0.102485,0.034102,-0.017516,0.083394,0.028430,...,0.000671,0.029643,-0.088444,0.015117,-0.033581,-0.013694,0.068252,-0.006680,-0.049313,-0.023267
6,-0.016129,0.010670,-0.127196,-0.060022,0.002483,-0.132768,0.043577,-0.024144,0.041696,0.071835,...,-0.007658,-0.047649,0.029271,0.007496,-0.091418,0.007005,-0.151710,-0.054534,-0.014743,0.024869
7,0.007923,-0.006203,0.024959,0.002276,0.025260,0.019183,0.042499,-0.012133,0.026818,0.072069,...,-0.009854,-0.051516,-0.006361,0.001558,-0.051776,-0.020458,-0.020660,-0.049717,-0.043439,0.030088
8,0.039911,-0.030511,0.028975,0.111232,0.048069,-0.013203,0.025693,-0.020383,-0.012060,0.054117,...,-0.016942,-0.012336,-0.000163,0.019659,-0.028355,-0.021113,-0.087264,-0.025237,-0.030626,0.030441
9,0.002746,-0.052481,-0.032023,0.034535,0.035409,-0.030983,0.096103,-0.017516,-0.050062,0.059148,...,-0.026650,-0.163632,0.014229,0.002525,-0.054069,-0.000184,-0.079935,0.038420,-0.063088,-0.002461


In [13]:
import plotly.express as px
px.line(data)